In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import gc

In [2]:
from google.colab import files
uploaded=files.upload()

Saving train.csv to train.csv


In [3]:
from google.colab import files
uploaded=files.upload()

Saving test.csv to test.csv


In [4]:
from google.colab import files
uploaded=files.upload()

Saving sample_submission.csv to sample_submission.csv


In [5]:
train= pd.read_csv('train.csv')
test =  pd.read_csv('test.csv')
ss = pd.read_csv('sample_submission.csv')


TARGET = 'congestion'
test[TARGET] = 0

#Turning time to DateTime
train['time'] = pd.to_datetime(train['time'])
test['time'] = pd.to_datetime(test['time'])
ss['time'] = test['time']

#Combining all location features 
for df in [train, test]:
    df['xydir'] = df['x'].astype(str) + '_' + df['y'].astype(str) + train['direction']
    df['xy'] = df['x'].astype(str) + '_' + df['y'].astype(str) 
ss['xydir'] = test['xydir']

In [6]:
train['test'] = False
test['test'] = True
data = pd.concat([train, test]).reset_index(drop=True)
del train, test; gc.collect()

92

In [7]:
data['day'] = data.time.dt.weekday
data['hour'] =data.time.dt.hour
data['minute'] = data.time.dt.minute
data['dhm'] = data['day'].astype(str) + '_' + data['hour'].astype(str) + '_' + data['minute'].astype(str)
data['hm'] = data['hour'].astype(str) + '_' + data['minute'].astype(str)
data['hm_xydir'] = data['hm'] + data['xydir']
data['hm_xy'] = data['hm'] + data['xydir']

In [8]:
################
#Global stats
################
#Find the min, max, median, variance, and mean for both 
#the x-y-direction and x-y for each unique hour-minute of the day.
#Modified from https://www.kaggle.com/code/packinman/tps-mar-2022-automl-pycaret-regression
FEATURES = []
for location_time in ['hm_xydir', 'hm_xy']:
    for stat in ['min','max','median','var','mean']:
        name = f'global_{location_time}_{stat}'
        stat = data.loc[data.test==False, [location_time, TARGET]].groupby([location_time]).agg(stat).to_dict()[TARGET]
        data[name] = data[location_time].map(stat)
        FEATURES.append(name)

In [9]:
###############################
#Lag Stats
###############################
#I found the mean, var, median, min, max, and 1 interval shift
# for 3/5/10 interval windows and expanding windows.  The intervals
# were every week and every day.
EXPANDING = ['roll', 'expand']
STATS = ['mean','var','median', 'min','max','shift']
TIMES = ['dhm','hm']
LOCATIONS = ['xydir'] 
LENGTHS = [3,5, 10]
i=0
for stat in STATS:
    for time in TIMES:
        for length in LENGTHS:
            for expanding in EXPANDING:
                print(i, end=", ")
                name = f'{time}_{stat}_{length}_{expanding}' 
                if stat == 'shift':
                    #Only shift once per time
                    if length==3:
                        name = f'{time}_{stat}_1'
                        data[name] = data.groupby([time, 'xydir'])[TARGET].apply(lambda x: x.shift())
                elif expanding =='roll':
                    data[name] = data.groupby([time, 'xydir'])[TARGET].apply(lambda x: x.shift().rolling(length,min_periods=1).agg(stat))
                else:
                    #Only use the expanding 1x per loop
                    if length==3:
                        name = f'{time}_{stat}_{expanding}'
                        data[name] = data.groupby([time, 'xydir'])[TARGET].apply(lambda x: x.shift().expanding(min_periods=1).agg(stat))
                FEATURES.append(name)
                i+=1

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 

In [10]:
data

,row_id,time,x,y,direction,congestion,xydir,xy,test,day,...,dhm_max_3_roll,dhm_max_expand,dhm_max_5_roll,dhm_max_10_roll,hm_max_3_roll,hm_max_expand,hm_max_5_roll,hm_max_10_roll,dhm_shift_1,hm_shift_1
0,0,1991-04-01 00:00:00,0,0,EB,70,0_0EB,0_0,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1991-04-01 00:00:00,0,0,NB,49,0_0NB,0_0,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1991-04-01 00:00:00,0,0,SB,24,0_0SB,0_0,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1991-04-01 00:00:00,0,1,EB,18,0_1EB,0_1,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1991-04-01 00:00:00,0,1,NB,60,0_1NB,0_1,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851170,851170,1991-09-30 23:40:00,2,3,NB,0,2_3NB,2_3,True,0,...,68.0,73.0,72.0,73.0,70.0,78.0,70.0,73.0,68.0,70.0
851171,851171,1991-09-30 23:40:00,2,3,NE,0,2_3NE,2_3,True,0,...,47.0,77.0,48.0,77.0,26.0,96.0,50.0,50.0,18.0,25.0
851172,851172,1991-09-30 23:40:00,2,3,SB,0,2_3SB,2_3,True,0,...,74.0,81.0,74.0,74.0,79.0,90.0,79.0,79.0,74.0,79.0
851173,851173,1991-09-30 23:40:00,2,3,SW,0,2_3SW,2_3,True,0,...,27.0,42.0,41.0,42.0,33.0,53.0,33.0,44.0,11.0,11.0


In [11]:
#Checking that the hm calculation works
msk = (data.xydir == '0_0NB') & (data.hm=='0_0')
FEATURES_TO_SEE = ['xydir','hm', 'dhm','time', TARGET, 'hm_max_5_roll', 'hm_max_10_roll', 'hm_shift_1']
data.loc[msk, FEATURES_TO_SEE].head(20)

,xydir,hm,dhm,time,congestion,hm_max_5_roll,hm_max_10_roll,hm_shift_1
1,0_0NB,0_0,0_0_0,1991-04-01,49,NaN,NaN,NaN
4681,0_0NB,0_0,1_0_0,1991-04-02,54,49.0,49.0,49.0
9361,0_0NB,0_0,2_0_0,1991-04-03,50,54.0,54.0,54.0
14041,0_0NB,0_0,3_0_0,1991-04-04,12,54.0,54.0,50.0
18721,0_0NB,0_0,4_0_0,1991-04-05,24,54.0,54.0,12.0
23401,0_0NB,0_0,5_0_0,1991-04-06,40,54.0,54.0,24.0
28081,0_0NB,0_0,6_0_0,1991-04-07,29,54.0,54.0,40.0
32761,0_0NB,0_0,0_0_0,1991-04-08,28,50.0,54.0,29.0
37441,0_0NB,0_0,1_0_0,1991-04-09,29,40.0,54.0,28.0
42121,0_0NB,0_0,2_0_0,1991-04-10,25,40.0,54.0,29.0


In [12]:
data.columns

Index(['row_id', 'time', 'x', 'y', 'direction', 'congestion', 'xydir', 'xy',
       'test', 'day', 'hour', 'minute', 'dhm', 'hm', 'hm_xydir', 'hm_xy',
       'global_hm_xydir_min', 'global_hm_xydir_max', 'global_hm_xydir_median',
       'global_hm_xydir_var', 'global_hm_xydir_mean', 'global_hm_xy_min',
       'global_hm_xy_max', 'global_hm_xy_median', 'global_hm_xy_var',
       'global_hm_xy_mean', 'dhm_mean_3_roll', 'dhm_mean_expand',
       'dhm_mean_5_roll', 'dhm_mean_10_roll', 'hm_mean_3_roll',
       'hm_mean_expand', 'hm_mean_5_roll', 'hm_mean_10_roll', 'dhm_var_3_roll',
       'dhm_var_expand', 'dhm_var_5_roll', 'dhm_var_10_roll', 'hm_var_3_roll',
       'hm_var_expand', 'hm_var_5_roll', 'hm_var_10_roll', 'dhm_median_3_roll',
       'dhm_median_expand', 'dhm_median_5_roll', 'dhm_median_10_roll',
       'hm_median_3_roll', 'hm_median_expand', 'hm_median_5_roll',
       'hm_median_10_roll', 'dhm_min_3_roll', 'dhm_min_expand',
       'dhm_min_5_roll', 'dhm_min_10_roll', 'hm_min_3_r

In [13]:
#Checking that the dhm calculation works
msk = (data.xydir == '0_0NB') & (data.dhm=='0_0_0')
FEATURES_TO_SEE = ['xydir','dhm','time', 'dhm_mean_3_roll','dhm_mean_expand','dhm_mean_5_roll', 'dhm_mean_10_roll']
data.loc[msk, FEATURES_TO_SEE]

,xydir,dhm,time,dhm_mean_3_roll,dhm_mean_expand,dhm_mean_5_roll,dhm_mean_10_roll
1,0_0NB,0_0_0,1991-04-01,NaN,NaN,NaN,NaN
32761,0_0NB,0_0_0,1991-04-08,49.000000,49.000000,49.000000,49.000000
65521,0_0NB,0_0_0,1991-04-15,38.500000,38.500000,38.500000,38.500000
98216,0_0NB,0_0_0,1991-04-22,37.666667,37.666667,37.666667,37.666667
130521,0_0NB,0_0_0,1991-04-29,28.000000,33.250000,33.250000,33.250000
162891,0_0NB,0_0_0,1991-05-06,31.000000,34.000000,34.000000,34.000000
195651,0_0NB,0_0_0,1991-05-13,31.000000,34.333333,31.400000,34.333333
228346,0_0NB,0_0_0,1991-05-20,37.333333,35.000000,33.600000,35.000000
261106,0_0NB,0_0_0,1991-05-27,34.000000,34.000000,31.800000,34.000000
293801,0_0NB,0_0_0,1991-06-03,30.000000,32.888889,32.600000,32.888889


In [14]:
data.to_parquet('all_rows.parquet')